In [ ]:
import time
import psutil
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from prometheus_client import start_http_server, Gauge

In [ ]:
cpu_threshold = 80  # CPU usage threshold (%)
memory_threshold = 70  # Memory usage threshold (%)
disk_threshold = 80  # Disk usage threshold (%)
traffic_threshold = 1000  # Network traffic threshold (bytes/s)

In [ ]:
# Create Prometheus metrics
cpu_usage_metric = Gauge('cpu_usage_percent', 'CPU Usage (%)')
memory_usage_metric = Gauge('memory_usage_percent', 'Memory Usage (%)')
disk_usage_metric = Gauge('disk_usage_percent', 'Disk Usage (%)')
network_traffic_metric = Gauge('network_traffic_bytes', 'Network Traffic (bytes)')

In [ ]:
def get_cpu_usage():
    # Function to get the current CPU usage as a percentage.
    return psutil.cpu_percent(interval=1)

In [ ]:
def get_memory_usage():
    # Function to get the current memory usage as a percentage.
    return psutil.virtual_memory().percent

In [ ]:
def get_disk_usage():
    # Function to get the current disk usage as a percentage.
    return psutil.disk_usage('/').percent

In [ ]:
def get_network_traffic():
    """
    Get the current network traffic in bytes.
    """
    return psutil.net_io_counters().bytes_sent + psutil.net_io_counters().bytes_recv

In [ ]:
def collect_system_metrics(data_file, sampling_interval):
    # Open the file in append mode
    with open(data_file, 'a') as f:
        # Write the header if the file is empty
        if f.tell() == 0:
            f.write('Timestamp, CPU Usage (%), Memory Usage (%), Disk Usage (%), Network Traffic (bytes)\n')

        # Start the data collection loop
        while True:
            # Get the current timestamp
            timestamp = int(time.time())

            # Retrieve the system metrics
            cpu_usage = get_cpu_usage()
            memory_usage = get_memory_usage()
            disk_usage = get_disk_usage()
            network_traffic = get_network_traffic()

            # Write the metrics to the file
            f.write(f'{timestamp}, {cpu_usage}, {memory_usage}, {disk_usage}, {network_traffic}\n')
            f.flush()  # Flush the buffer to ensure immediate write to the file

            # Update Prometheus metrics
            cpu_usage_metric.set(cpu_usage)
            memory_usage_metric.set(memory_usage)
            disk_usage_metric.set(disk_usage)
            network_traffic_metric.set(network_traffic)

            # Check if any metric exceeds the defined thresholds
            if cpu_usage > cpu_threshold or memory_usage > memory_threshold or disk_usage > disk_threshold:
                # Raise an alert
                alert_message = f"Anomaly detected at {timestamp}: CPU Usage: {cpu_usage}%, Memory Usage: {memory_usage}%, Disk Usage: {disk_usage}%"
                send_alert(alert_message)

            # Sleep for the specified sampling interval
            time.sleep(sampling_interval)

In [ ]:
def send_alert(message):
    # Replace this placeholder with your actual alerting mechanism
    print(f"ALERT: {message}")

In [ ]:
def visualize_system_metrics(data_file):
    # Read the data from the file into a pandas DataFrame
    df = pd.read_csv(data_file)

    # Convert the timestamp column to a datetime type
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')

    # Visualize the CPU, Memory, and Disk Usage over time
    plt.figure(figsize=(12, 6))
    plt.plot(df['Timestamp'], df[' CPU Usage (%)'], label='CPU Usage')
    plt.plot(df['Timestamp'], df[' Memory Usage (%)'], label='Memory Usage')
    plt.plot(df['Timestamp'], df[' Disk Usage (%)'], label='Disk Usage')
    plt.xlabel('Time')
    plt.ylabel('Usage (%)')
    plt.title('System Metrics')
    plt.legend()
    plt.show()

In [ ]:
def apply_anomaly_detection(data_file):
    # Read the data from the file into a pandas DataFrame
    df = pd.read_csv(data_file)

    # Define the features for anomaly detection
    features = ['CPU Usage (%)', 'Memory Usage (%)', 'Disk Usage (%)']

    # Fit the Isolation Forest model to the data
    model = IsolationForest(contamination='auto', random_state=42)
    model.fit(df[features])

    # Predict the anomalies
    predictions = model.predict(df[features])

    # Get the indices of the anomalies
    anomaly_indices = df.index[predictions == -1].tolist()

    if anomaly_indices:
        print("Anomalies detected at the following indices:")
        print(anomaly_indices)
        # Raise an alert for each anomaly detected
        for index in anomaly_indices:
            alert_message = f"Anomaly detected at index {index}: {df.loc[index]}"
            send_alert(alert_message)
    else:
        print("No anomalies detected.")

In [ ]:
# Define the path to the file where you want to store the collected data
data_file = 'system_metrics.csv'

In [ ]:
# Read the data from the file into a pandas DataFrame
df = data_file

In [ ]:
# Define the sampling interval in seconds
sampling_interval = 5

In [ ]:
# Start the Prometheus HTTP server
start_http_server(8000)  # Replace with the desired port number

In [ ]:
# Call the function to start collecting system metrics and store them in the data file
collect_system_metrics(data_file, sampling_interval)

In [ ]:
# Visualize the system metrics
visualize_system_metrics(data_file)

In [ ]:
# Apply anomaly detection on the system metrics
apply_anomaly_detection(data_file)

In [ ]:
visualize_system_metrics(data_file)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['Timestamp'], df[' CPU Usage (%)'])
plt.xlabel('Timestamp')
plt.ylabel('CPU Usage (%)')
plt.title('CPU Usage over Time')
plt.grid(True)
plt.show()